In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2023-08-25 11:11:08.098229: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-25 11:11:08.125578: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-25 11:11:08.322259: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-25 11:11:08.323294: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 11:11:09.158807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
tf.__version__

'2.13.0'

In [3]:
dataset = pd.read_excel('customer_churn_large_dataset.xlsx')


In [4]:
dataset.head()

,CustomerID,Name,Age,Gender,Location,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB,Churn
0,1,Customer_1,63,Male,Los Angeles,17,73.36,236,0
1,2,Customer_2,62,Female,New York,1,48.76,172,0
2,3,Customer_3,24,Female,Los Angeles,5,85.47,460,0
3,4,Customer_4,36,Female,Miami,3,97.94,297,1
4,5,Customer_5,46,Female,Miami,19,58.14,266,0


In [5]:
X=dataset.iloc[:,2:-1].values
y=dataset.iloc[:,-1].values

In [6]:
print(X)

[[63 'Male' 'Los Angeles' 17 73.36 236]
 [62 'Female' 'New York' 1 48.76 172]
 [24 'Female' 'Los Angeles' 5 85.47 460]
 ...
 [64 'Male' 'Chicago' 17 96.11 251]
 [51 'Female' 'New York' 20 49.25 434]
 [27 'Female' 'Los Angeles' 19 76.57 173]]


In [7]:
print(y)

[0 0 0 ... 1 1 1]


In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   CustomerID                  100000 non-null  int64  
 1   Name                        100000 non-null  object 
 2   Age                         100000 non-null  int64  
 3   Gender                      100000 non-null  object 
 4   Location                    100000 non-null  object 
 5   Subscription_Length_Months  100000 non-null  int64  
 6   Monthly_Bill                100000 non-null  float64
 7   Total_Usage_GB              100000 non-null  int64  
 8   Churn                       100000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 6.9+ MB


# Label Encoding the "Gender" column

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 1] = le.fit_transform(X[:, 1])

In [10]:
print(X)

[[63 1 'Los Angeles' 17 73.36 236]
 [62 0 'New York' 1 48.76 172]
 [24 0 'Los Angeles' 5 85.47 460]
 ...
 [64 1 'Chicago' 17 96.11 251]
 [51 0 'New York' 20 49.25 434]
 [27 0 'Los Angeles' 19 76.57 173]]


# One Hot Encoding the "Location" column

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [12]:
print(X)

[[0.0 0.0 1.0 ... 17 73.36 236]
 [0.0 0.0 0.0 ... 1 48.76 172]
 [0.0 0.0 1.0 ... 5 85.47 460]
 ...
 [1.0 0.0 0.0 ... 17 96.11 251]
 [0.0 0.0 0.0 ... 20 49.25 434]
 [0.0 0.0 1.0 ... 19 76.57 173]]


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 49)

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler


In [16]:
ann = tf.keras.models.Sequential()

2023-08-25 11:11:27.227943: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-25 11:11:27.229204: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [17]:
def lr_schedule(epoch):
    initial_lr = 0.001
    decay_rate = 0.9
    decay_step = 10
    new_lr = initial_lr * (decay_rate ** (epoch // decay_step))
    return new_lr

In [18]:
ann = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train.shape[1]),
    tf.keras.layers.Dense(units=64, activation='relu'),
    Dropout(0.5),
    tf.keras.layers.Dense(units=32, activation='relu'),
    Dropout(0.5),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])


In [19]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(lr_schedule)


In [21]:
history = ann.fit(X_train, y_train, batch_size=64, epochs=50, validation_split=0.1, callbacks=[early_stopping, lr_scheduler])

Epoch 1/50
1125/1125 [==============================] - 2s 1ms/step - loss: 0.6950 - accuracy: 0.5019 - val_loss: 0.6931 - val_accuracy: 0.4950 - lr: 0.0010
Epoch 2/50
1125/1125 [==============================] - 1s 1ms/step - loss: 0.6934 - accuracy: 0.5005 - val_loss: 0.6933 - val_accuracy: 0.4995 - lr: 0.0010
Epoch 3/50
1125/1125 [==============================] - 1s 1ms/step - loss: 0.6934 - accuracy: 0.5014 - val_loss: 0.6931 - val_accuracy: 0.4972 - lr: 0.0010
Epoch 4/50
1125/1125 [==============================] - 1s 1ms/step - loss: 0.6933 - accuracy: 0.4974 - val_loss: 0.6931 - val_accuracy: 0.5015 - lr: 0.0010
Epoch 5/50
1125/1125 [==============================] - 1s 1ms/step - loss: 0.6933 - accuracy: 0.5040 - val_loss: 0.6932 - val_accuracy: 0.4942 - lr: 0.0010
Epoch 6/50
1125/1125 [==============================] - 1s 1ms/step - loss: 0.6932 - accuracy: 0.5019 - val_loss: 0.6932 - val_accuracy: 0.4971 - lr: 0.0010
Epoch 7/50
1125/1125 [==============================] - 1s